<a href="https://colab.research.google.com/github/AlejandroMllo/Machine-Learning-Algorithms/blob/master/Evolutionary%20Algorithms/StrengthParetoEvolutionaryAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Strength Pareto Evolutionary Algorithm (SPEA)

## SPEA *(Zitler & Thiele, 1998)*


## SPEA2 *(Zitler et al., 2001)*

### Example using SPEA2

Solution of the multiobjective optimization problem SCH, problem 1 described in (Deb et al., 2002), given by:

$$\\ \min   f_1(x) + f_2(x) $$

$where$
      $$  f_1(x) = \Sigma_1^n (x_i^2) $$
      $$ f_2(x) = \Sigma_1^n (x_i - 2)^2 $$
     
$subject\ to$
    $$ -10 \leq x_i \leq 10, n = 1\\$$

$whose\ optimal\ value\ is:\ x \in [0, 2].$

using SPEA2.

In [0]:
# Required Libraries
import numpy as np

from random import random, randint

In [0]:
# Objective Functions

def f1(x):
  
  x = x**2
  return np.sum(x)


def f2(x):
  
  x = (x - 2)**2
  return np.sum(x)

In [0]:
# Helper Functions

def decode(bitstring, search_space, bits_per_param):
  """
  _genotype_ to _fenotype_ conversion.
  """
  
  vector = np.array([])
  
  for index, bound in enumerate(search_space):
    
    off, sumation = index*bits_per_param, 0.0
    # param = list(reversed(bitstring[off : off + bits_per_param]))
    param = reversed(bitstring[off : off + bits_per_param])
    
    for j in range(len(param)):
      sumation += float((1.0 if param[j] == '1' else 0.0) * (2.0 ** j))
      
    minimum, maximum = bound
    
    new_element = \
      minimum + ( (maximum - minimum)/((2.0**bits_per_param) - 1.0) ) * sumation
    vector = np.append(vector, new_element)
    
  return vector


def random_bitstring(num_bits):
  
  bits = ['1' if random() < 0.5 else '0' for _ in range(num_bits)]
  return ''.join(bits)


def euclidean_distance(v1, v2):
  return np.linalg.norm(v1 - v2, 2)

In [0]:
# Variation Operations

def point_mutation(bitstring, rate=None):
  
  if rate is None:
    rate = 1.0 / len(bitstring)
    
  child = ''
  
  for i in range(len(bitstring)):
    bit = bitstring[i]
    alternate_bit = 1 if bit == '1' else 0
    child += alternate_bit if random() < rate else bit
    
  return child


def binary_tournament(pop):
  
  pop_len = len(pop)
  i, j = randint(0, pop_len - 1), randint(0, pop_len - 1)
  while i == j:  # Used to guarantee i and j are different.
    j = randint(0, pop_len - 1)
    
  return pop[i] if pop[i].fitness() < pop[j].fitness() else pop[j]


def crossover(parent1, parent2, rate):
  
  if random() >= rate:
    return '' + parent1
  
  child = ''
  for i in range(len(parent1)):
    
    child += parent1[i] if random() < 0.5 else parent2[i]
    
  return child

In [0]:
# Reproduction

def reproduce(selected, pop_size, p_cross):
  
  children = np.array([])
  for index, ind1 in enumerate(selected):
    ind2 = selected[index + 1] if index % 2 == 0 else selected[index - 1]
    if index == ( len(selected) - 1 ):
      ind2 = selected[0]
      
    child = Individual()
    child.set_bitstring( crossover(ind1.bitstring(), ind2.bitstring(), p_cross) )
    child.set_bitstring( point_mutation(child.bitstring()) )
    
    children = np.append(children, child)
    
    if len(children) > pop_size:
      break
      
  return children

In [0]:
# Individuals Evaluation

def calculate_objective(pop, search_space, bits_per_param):
  
  for ind in pop:
    
    ind.set_vector( decode(ind.bitstring(), search_space, bits_per_param) )
    objectives = [
        f1(ind.vector())
      , f2(ind.vector())
    ]
    ind.set_objectives(objectives)
    

def is_dominant(ind1, ind2):
  
  ind1_objectives = ind1.objectives()
  ind2_objectives = ind2.objectives()
  
  for i in range(len(ind1_objectives)):
    if ind1_objectives[i] > ind2_objectives[i]:
      return False
    
  return True


def weighted_sum(ind):
  
  ind_objectives = ind.objectives()
  return np.sum(ind_objectives)


def calculate_dominated(pop):
  
  for ind in pop:
    dominated_set = [
        dominated_ind for dominated_ind in ind.dominated_set()
                      if is_dominant(ind, dominated_ind) and ind != dominated_ind
    ]
    ind.set_dominated_set(dominated_set)
    

def calculate_raw_fitness(individual, pop):
  
  raw_individuals = [
      len(ind.dominated_set()) for ind in pop if is_dominant(ind, individual)
  ]    
      
  return sum(raw_individual)


def calculate_density(individual, pop):
  
  for ind in pop:
    ind.set_distance(
        euclidean_distance(individual.objectives(), ind.objectives())
    )
  
  # Sorted by increasing distance.
  sorted_dist = np.sort(
    [ind.distance() for ind in pop]
  )
  
  k = np.sqrt(len(pop))
  distance_sought = 1.0 / (sorted_dist[k] + 2.0)
  
  return distance_sought


def calculate_fitness(pop, archive, search_space, bits_per_param):
  
  calculate_objectives(pop, search_space, bits_per_param)
  
  union = np.append(pop, archive)
  
  calculate_dominated(union)
  
  for ind in union:
    ind.set_raw_fitness( calculate_raw_fitness(ind, union) )
    ind.set_density( calculate_density(ind, union) )

In [0]:
# Environmental Selection



In [0]:
class Individual:
  
  def __init__():
        
    self.__bitstring = None
    self.__vector = None
    
    self.__objectives = None
    
    self.__dominated_set = None
    self.__distance = None
    
    self.__raw_fitness = None
    self.__density = None
    
  # Getters
  def bitstring():
    return self.__bitstring
  
  def vector():
    return  self.__vector
  
  def objectives():
    return self.__objectives
  
  def dominated_set():
    return self.__dominated_set
  
  def distance():
    return self.__distance
  
  def raw_fitness():
    return self.__raw_fitness
  
  def density():
    return self.__density
  
  def fitness():
    return self.density() + self.raw_fitness()
  
  # Setters
  def set_bitstring(bitstring):
    self.__bitstring = bitstring
    
  def set_vector(vector):
    self.__vector = vector
    
  def set_objectives(objectives):
    self.__objectives = objectives
    
  def set_dominated_set(dom_set):
    self.__dominated_set = dom_set
    
  def set_distance(dist):
    self.__distance = dist
  
  def set_raw_fitness(raw_fitness):
    self.__raw_fitness = raw_fitness
    
  def set_density(density):
    self.__density = density
  

## Referencias

- (Zitzler & Thiele, 1998) Zitzler, E., & Thiele, L. (1998). An evolutionary algorithm for
multiobjective optimization: The strength pareto approach. TIK-report, 43.
- (Zitler et al., 2001) Zitzler, E., Laumanns, M., & Thiele, L. (2001). SPEA2: Improving the strength Pareto evolutionary algorithm. TIK-report, 103.
- (Deb et al., 2018)   K. Deb and A. Pratap and S. Agarwal and T. Meyarivan, "A Fast and Elitist Multiobjective Genetic Algorithm: NSGA–II", IEEE Transactions on Evolutionary Computation, 2002. 